In [2]:
import pandas as pd
import numpy as np
import os
import re
import random
import json
# from sentence_transformers import SentenceTransformer
from context_doc_embed import *

data_dir = "/Users/Shiquan/Documents/WPI Courses/Intro of DS/Project/Datasets/NIPS 2019"
# author_path = os.path.join(data_dir, "authors.csv")
paper_path = os.path.join(data_dir, "papers.csv")
# author_df = pd.read_csv(author_path)
paper_df = pd.read_csv(paper_path)
print("Shape before removing null text ones: ", paper_df.shape)  # author_df.shape,
if len(paper_df) == paper_df.title.nunique():
    print("No dupliate title in the data.")
    paper_df["id"] = range(1, len(paper_df) + 1)
else:
    print("Dupliate titles in the data!")
print(paper_df.source_id.nunique(), paper_df.title.nunique())

paper_df.dropna(subset=["full_text"], inplace=True)
print("Shape after removing null text ones: ", paper_df.shape)

paper_df.head()

/Users/Shiquan/opt/anaconda3/envs/ds501/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Shape before removing null text ones:  (9680, 5)
No dupliate title in the data.
4522 9680
Shape after removing null text ones:  (9677, 6)


,source_id,year,title,abstract,full_text,id
0,27,1987,Bit-Serial Neural Networks,NaN,573 \n\nBIT - SERIAL NEURAL NETWORKS \n\nAlan...,1
1,63,1987,Connectivity Versus Entropy,NaN,1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser S...,2
2,60,1987,The Hopfield Model with Multi-Level Neurons,NaN,278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...,3
3,59,1987,How Neural Nets Work,NaN,442 \n\nAlan Lapedes \nRobert Farber \n\nThe...,4
4,69,1987,Spatial Organization of Neural Networks: A Pro...,NaN,740 \n\nSPATIAL ORGANIZATION OF NEURAL NEn...,5


In [6]:
# sample some papers as our target papers

target_paper_num = 5

random.seed(42)
target_ids = random.sample(paper_df['id'].tolist(), target_paper_num)
target_ids

[1825, 410, 4508, 4013, 3658]

In [ ]:
## Use Google Colab for getting embeddings
from sentence_transformers import SentenceTransformer, util
import json
import pickle
from time import time
from tqdm import tqdm

sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
save_dir = '/content/drive/MyDrive/Colab/Colab data/NIPS 2019'
num_per_file = 1000

paper_list = []

st_time = time()

for i, a_paper in tqdm(paper_df.iterrows()):
    paper_info = dict()
    paper_info['id'] = a_paper['id']
    paper_info['title'] = a_paper['title']
    paper_info['sentence'] = get_norm_sentences_from_text(a_paper['full_text'])
    paper_info['sentence_embed'] = [sbert_model.encode(x) for x in paper_info['sentence']]
    paper_info['doc_embed'] = pd.DataFrame(paper_info['sentence_embed']).apply(np.mean, axis=0).to_numpy()
    paper_list.append(paper_info)
    if (i > 0) & (i % num_per_file == 0)| (i == max(paper_df.index)):
        file_index = i // num_per_file if i != max(paper_df.index) else ((i // num_per_file) +1)
        paper_embeddings = {"paper": paper_list}
        processed_paper_file_name = f"processed_nips_paper_{file_index}.pickle"
        with open(os.path.join(save_dir, processed_paper_file_name), "wb") as pkl:
            pickle.dump(paper_embeddings, pkl)
        paper_list = []
        
## It takes 5 hours on getting the embeddings

# paper_embeddings = {"paper": paper_list}



# processed_paper_file_name = "processed_nips_paper.json"
# with open(processed_paper_file_name, "w") as fp:
#     json.dump(paper_embeddings, fp, indent=2)

    

In [ ]:

file_dir = "/Users/Shiquan/Library/CloudStorage/GoogleDrive-s.quanhe@gmail.com/My Drive/Colab/Colab data/NIPS 2019"
file_list = ["processed_nips_paper_" + str(x) + ".pickle" for x in range(1, 11)]
file_list = [os.path.join(file_dir, x) for x in file_list]

save_dir2 = '/Users/Shiquan/Documents/WPI Courses/Intro of DS/Project/Datasets/NIPS 2019'

merge_pickles(file_path_list=file_list, 
              new_file_path=os.path.join(save_dir2, 'sentence_doc_embeddings.pickle'))

In [ ]:
with open(os.path.join(save_dir2, 'sentence_doc_embeddings.pickle'), 'rb') as pkl:
            sent_doc_embed = pickle.load(pkl)

print(len(sent_doc_embed))

In [ ]:
## Generate a smaller file with only id, title and doc embeddings

doc_embed_list = []

for one_doc in sent_doc_embed:
    del one_doc['sentence']
    del one_doc['sentence_embed']
    doc_embed_list.append(one_doc)


with open(os.path.join(save_dir2, 'doc_embeddings.pickle'), "wb") as pkl:
    pickle.dump(doc_embed_list, pkl)

In [ ]:
with open(os.path.join(save_dir2, 'doc_embeddings.pickle'), 'rb') as pkl:
    doc_embed_list = pickle.load(pkl)

doc_embed_df = pd.DataFrame(doc_embed_list)

print(doc_embed_df.shape)
doc_embed_df.head(2) 

In [ ]:
target_id = target_ids[0]
# non_null_embed_indexes = [x for x in range(len(doc_embed_list)) if x!= 6104]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

## 6104 has null full text

In [ ]:
target_id = target_ids[1]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)



In [ ]:
target_id = target_ids[2]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

In [ ]:
target_id = target_ids[3]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

In [ ]:
target_id = target_ids[4]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

In [ ]:
target_title1 = "Computing with Finite and Infinite Networks"
target_text1 = paper_df.loc[paper_df['title']==target_title1, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=bert_found1_df.iloc[0]['full_text'], reference=target_text1))

print(get_mean_rouge(candidate_df=bert_found1_df, reference=target_text1))

print(bert_found1_df.iloc[0]['title'])

In [ ]:
target_id = target_ids[0]
# non_null_embed_indexes = [x for x in range(len(doc_embed_list)) if x!= 6104]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
bert_found1_df = get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)
bert_found1_df = bert_found1_df.merge(paper_df[['id', 'full_text']], 
                                      on='id', how='left')
bert_found1_df


In [ ]:
target_id = target_ids[1]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
bert_found2_df = get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

bert_found2_df = bert_found2_df.merge(paper_df[['id', 'full_text']], 
                                      on='id', how='left')
bert_found2_df

In [ ]:
target_title2 = 'Integrated Segmentation and Recognition of Hand-Printed Numerals'
target_text2 = paper_df.loc[paper_df['title']==target_title2, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=bert_found2_df.iloc[0]['full_text'], reference=target_text2))

print(get_mean_rouge(candidate_df=bert_found2_df, reference=target_text2))

print(bert_found2_df.iloc[0]['title'])

In [14]:
target_id = target_ids[2]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
bert_found3_df = get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

bert_found3_df = bert_found3_df.merge(paper_df[['id', 'full_text']], 
                                      on='id', how='left')
bert_found3_df

rouge.rouge.Rouge

In [15]:
target_title3 = 'Accelerated Training for Matrix-norm Regularization: A Boosting Approach'
target_text3 = paper_df.loc[paper_df['title']==target_title3, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=bert_found3_df.iloc[0]['full_text'], reference=target_text3))

print(get_mean_rouge(candidate_df=bert_found3_df, reference=target_text3))

print(bert_found3_df.iloc[0]['title'])

{'rouge-1': {'r': 0.2806236080178174,
  'p': 0.16578947368421051,
  'f': 0.2084367198966123},
 'rouge-2': {'r': 0.05944055944055944,
  'p': 0.03426432479124676,
  'f': 0.04347031499563712},
 'rouge-l': {'r': 0.2639198218262806,
  'p': 0.15592105263157896,
  'f': 0.19602977200579344}}

In [16]:
target_id = target_ids[3]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
bert_found4_df = get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

bert_found4_df = bert_found4_df.merge(paper_df[['id', 'full_text']], 
                                      on='id', how='left')
bert_found4_df

0.2084367198966123

In [4]:
target_title4 = 'Sodium entry efficiency during action potentials: A novel single-parameter family of Hodgkin-Huxley models'
target_text4 = paper_df.loc[paper_df['title']==target_title4, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=bert_found4_df.iloc[0]['full_text'], reference=target_text4))

print(get_mean_rouge(candidate_df=bert_found4_df, reference=target_text4))

print(bert_found4_df.iloc[0]['title'])

[{'rouge-1': {'r': 0.2806236080178174,
   'p': 0.16578947368421051,
   'f': 0.2084367198966123},
  'rouge-2': {'r': 0.05944055944055944,
   'p': 0.03426432479124676,
   'f': 0.04347031499563712},
  'rouge-l': {'r': 0.2639198218262806,
   'p': 0.15592105263157896,
   'f': 0.19602977200579344}}]

In [19]:
target_id = target_ids[4]

print(doc_embed_df.loc[doc_embed_df.id==target_id, 'title'])
bert_found5_df = get_most_similar_docs_from_list(
        target_id=target_id, 
        embed_list=doc_embed_list)

bert_found5_df = bert_found5_df.merge(paper_df[['id', 'full_text']], 
                                      on='id', how='left')
bert_found5_df

2.0

In [ ]:
## Print the ROUGEs from top 10 relevant paper for a target paper

bert_found_df = bert_found5_df
target_text = target_text5
rouge_lst = [get_rouge_metrics(candi_text=bert_found_df.iloc[i]['full_text'], reference=target_text) for i in range(0, 10)]

for i, a_rouge in enumerate(rouge_lst):
    print(bert_found_df.iloc[i]['title'])
    print(a_rouge[0]['rouge-1']['f'], a_rouge[0]['rouge-l']['f'], '\n')

In [ ]:
## Get the keywords for all the doc in NIPS data
from tqdm import tqdm

paper_list = []
consider_top_n = 50
save_dir2 = '/Users/Shiquan/Documents/WPI Courses/Intro of DS/Project/Datasets/NIPS 2019'

for i, a_paper in tqdm(paper_df.iterrows()):
    paper_info = dict()
    paper_info['id'] = a_paper['id']
    paper_info['title'] = a_paper['title']
    paper_info['full_text'] = a_paper['full_text']
    keyword_info = get_top_keywords(text=a_paper['full_text'], top_n=consider_top_n)
    paper_info['key_words'] = keyword_info['key_word']
    paper_info['rank_scores'] = keyword_info['rank_score']
    paper_info['doc_len'] = get_word_count(a_paper['full_text'])
    paper_list.append(paper_info)

paper_keywords = {"paper": paper_list}
with open(os.path.join(save_dir2, "nips_paper_key_words.pickle"), "wb") as pkl:
    pickle.dump(paper_keywords, pkl)



In [ ]:
with open(os.path.join(save_dir2, "nips_paper_key_words.pickle"), "wb") as pkl:
    pickle.dump(paper_keywords, pkl)


In [20]:
with open(os.path.join(save_dir2, "nips_paper_key_words.pickle"), 'rb') as pkl:
    paper_keywords = pickle.load(pkl)

paper_keywords_list = paper_keywords['paper']
print(len(paper_keywords_list))

1

In [ ]:
target_id=target_ids[0]
tr_found1_df = get_most_similar_docs_from_list_textrank(
    target_id=target_id,
    keyword_info_list=paper_keywords_list,
    target_text= None,
    top_n= 10,
    top_n_keywords= 20,
)
tr_found1_df

In [ ]:
target_title = "Computing with Finite and Infinite Networks"
target_text = paper_df.loc[paper_df['title']==target_title, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=tr_found1_df.iloc[0]['full_text'], reference=target_text))
print(get_mean_rouge(candidate_df=tr_found1_df, reference=target_text))

In [ ]:
target_id=target_ids[1]
tr_found2_df = get_most_similar_docs_from_list_textrank(
    target_id=target_id,
    keyword_info_list=paper_keywords_list,
    target_text= None,
    top_n= 10,
    top_n_keywords= 20,
)

In [ ]:
target_title2 = "Integrated Segmentation and Recognition of Hand-Printed Numerals"
target_text2 = paper_df.loc[paper_df['title']==target_title2, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=tr_found2_df.iloc[0]['full_text'], reference=target_text2))

print(get_mean_rouge(candidate_df=tr_found2_df, reference=target_text2))

print(tr_found2_df.iloc[0]['title'])

In [ ]:
target_id=target_ids[2]
tr_found3_df = get_most_similar_docs_from_list_textrank(
    target_id=target_id,
    keyword_info_list=paper_keywords_list,
    target_text= None,
    top_n= 10,
    top_n_keywords= 20,
)

In [ ]:
target_title3 = "Accelerated Training for Matrix-norm Regularization: A Boosting Approach"
target_text3 = paper_df.loc[paper_df['title']==target_title3, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=tr_found3_df.iloc[0]['full_text'], reference=target_text3))

print(get_mean_rouge(candidate_df=tr_found3_df, reference=target_text3))

print(tr_found3_df.iloc[0]['title'])

In [ ]:
target_id=target_ids[3]
tr_found4_df = get_most_similar_docs_from_list_textrank(
    target_id=target_id,
    keyword_info_list=paper_keywords_list,
    target_text= None,
    top_n= 10,
    top_n_keywords= 20,
)

In [ ]:
target_title4 = "Sodium entry efficiency during action potentials: A novel single-parameter family of Hodgkin-Huxley models"
target_text4 = paper_df.loc[paper_df['title']==target_title4, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=tr_found4_df.iloc[0]['full_text'], reference=target_text4))

print(get_mean_rouge(candidate_df=tr_found4_df, reference=target_text4))

print(tr_found4_df.iloc[0]['title'])

In [ ]:
target_id=target_ids[4]
tr_found5_df=get_most_similar_docs_from_list_textrank(
    target_id=target_id,
    keyword_info_list=paper_keywords_list,
    target_text= None,
    top_n= 10,
    top_n_keywords= 20,
)

In [ ]:
target_title5 = "Modeling Social Annotation Data with Content Relevance using a Topic Model"
target_text5 = paper_df.loc[paper_df['title']==target_title5, 'full_text'].item()
print(get_rouge_metrics(
    candi_text=tr_found5_df.iloc[0]['full_text'], reference=target_text5))

print(get_mean_rouge(candidate_df=tr_found5_df, reference=target_text5))

print(tr_found5_df.iloc[0]['title'])